In [9]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

In [6]:
df = pd.read_csv('Base_covid.csv', index_col=0)
#available_indicators = df['Indicator Name'].unique()
available_indicators = ['a','b','c']

In [3]:
dados = {'medicamento':[], 'positivo':[], 'negativo':[]}
for c in df.columns[9:]:
    dados['medicamento'].append(c)
    dados['positivo'].append((df[c] == 1).sum())
    dados['negativo'].append((df[c] == 0).sum())
dados = pd.DataFrame(dados)

In [14]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

data = [go.Bar(y=dados['medicamento'].head(20),
               x=dados['positivo'].head(20),
               orientation='h',
               name='Positivos',
               hoverinfo='x',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=dados['medicamento'].head(20),
               x=-dados['negativo'].head(20),
               orientation='h',
               name='Negativos',
               text=dados['negativo'],
               hoverinfo='text',
               marker=dict(color='orange')
               )]
fig = go.Figure(data=data)

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='a'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='b'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(figure=fig, id='crossfilter-indicator-scatter')
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(figure=fig, id='crossfilter-indicator-scatter2')
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=dados['positivo'].min(),
        max=dados['positivo'].max(),
        value=dados['positivo'].max(),
        marks={str(year): str(year) for year in dados['positivo'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


#@app.callback(
#    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
#    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
#     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
#     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
#     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
#     dash.dependencies.Input('crossfilter-year--slider', 'value')])
#def update_graph(xaxis_column_name, yaxis_column_name,
#                 xaxis_type, yaxis_type,
#                 year_value):
#    dff = df[df['Year'] == year_value]

#    return {
#        'data': [dict(
#            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
#            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
#            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
#            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
#            mode='markers',
#            marker={
#                'size': 25,
#                'opacity': 0.7,
#                'color': 'orange',
#                'line': {'width': 2, 'color': 'purple'}
#            }
#        )],
#        'layout': dict(
#            xaxis={
#                'title': xaxis_column_name,
#                'type': 'linear' if xaxis_type == 'Linear' else 'log'
#            },
#            yaxis={
#                'title': yaxis_column_name,
#                'type': 'linear' if yaxis_type == 'Linear' else 'log'
#            },
#            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
#            height=450,
#            hovermode='closest'
#        )
#    }


#def create_time_series(dff, axis_type, title):
#    return {
#        'data': [dict(
#            x=dff['Year'],
#            y=dff['Value'],
#            mode='lines+markers'
#        )],
#        'layout': {
#            'height': 225,
#            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
#            'annotations': [{
#                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
#                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
#                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
#                'text': title
#            }],
#            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
#            'xaxis': {'showgrid': False}
#        }
#    }


#@app.callback(
#    dash.dependencies.Output('x-time-series', 'figure'),
#    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
#     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
#     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
#def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
#    country_name = hoverData['points'][0]['customdata']
#    dff = df[df['Country Name'] == country_name]
#    dff = dff[dff['Indicator Name'] == xaxis_column_name]
#    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
#    return create_time_series(dff, axis_type, title)


#@app.callback(
#    dash.dependencies.Output('y-time-series', 'figure'),
#    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
#     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
#     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
#def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
#    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
#    dff = dff[dff['Indicator Name'] == yaxis_column_name]
#    return create_time_series(dff, axis_type, yaxis_column_name)

In [15]:
app.run_server(mode="inline", debug=True)